In [25]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import preprocess
import importlib
import features

In [26]:
training_folder = "../shared_data/training_mini"

In [27]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(training_folder,90)

10	 Dateien wurden geladen.


In [28]:
samples_dict = {}
for label, id_, channel, datum, freq, ref in zip(eeg_labels, ids, channels, data, sampling_frequencies, reference_systems):  
    entry = {
        "channel": channel,
        "data": datum,
        "sampling_frequency": freq,
        "reference_system": ref,
        "label": label}
    samples_dict[id_] = entry

In [29]:
# Seperate into seizures and no seizures for feature test reasons 
with_seizure = []
no_seizure = []

for id_, entry in samples_dict.items():
    label = entry["label"]
    if label[0] == 1:
        with_seizure.append(id_)
    else:
        no_seizure.append(id_)

In [30]:
def prefilter(data, fs, notch_freq = 60, Q = 30.0):
    b, a = sig.iirnotch(notch_freq, Q, fs)
    filtered_signal = sig.filtfilt(b,a,data, axis =1)
    return filtered_signal

In [31]:
importlib.reload(preprocess)
%matplotlib inline
example_id = list(samples_dict.keys())[0]
entry = samples_dict[example_id]
signals = entry["data"]  # shape: (n_channels, n_samples)
fs = entry["sampling_frequency"]
ch = entry["channel"] # list len n channels
ref = entry["reference_system"]
preprocessed = preprocess.process_without_mne(signals, fs, ch,ref,lowest_sampling)
psds = []
for signal in preprocessed:
    freqs, psd = sig.welch(signal, fs)
    psds.append(psd)
    
avg_psd = np.mean(psds, axis=0)

plt.figure(figsize=(8, 4))
plt.semilogy(freqs, avg_psd)
plt.title(f"Avg PSD for ID: {id_} | Seizure: {entry['label'][0] == 1} | Reference System: {entry['reference_system']}")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density (V²/Hz)")
plt.grid(True)
plt.tight_layout()
plt.show()

      
'''
for id_, entry in samples_dict.items():
    signals = entry["data"]  # shape: (n_channels, n_samples)
    fs = entry["sampling_frequency"]
    
    filtered_signals = prefilter(np.array(signals), fs, notch_freq = 60.0)

    psds = []
    for signal in filtered_signals:
        freqs, psd = sig.welch(signal, fs)
        psds.append(psd)
    
    avg_psd = np.mean(psds, axis=0)

    plt.figure(figsize=(8, 4))
    plt.semilogy(freqs, avg_psd)
    plt.title(f"Avg PSD for ID: {id_} | Seizure: {entry['label'][0] == 1} | Reference System: {entry['reference_system']}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density (V²/Hz)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
'''

TypeError: process_without_mne() missing 1 required positional argument: 'lowest_sampling'

In [9]:
importlib.reload(features)

print(features.feature_extraction(preprocessed,fs).shape)


(19, 12)
